# Análise exploratória de dados

In [1]:
import pandas as pd

## 1. Descrição dos dados

#### Carregando os dados

In [2]:
df = pd.read_csv('../data/raw/titanic.csv')
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


#### Qtd de instâncias e variáveis

In [3]:
df.shape

(891, 15)

In [4]:
instancias, variaveis = df.shape
print(f"Numero de instâncias: {instancias}")
print(f"Numero de variáveis: {variaveis}")

Numero de instâncias: 891
Numero de variáveis: 15


#### Tipos das variáveis

In [5]:
df.dtypes

survived         int64
pclass           int64
sex             object
age            float64
sibsp            int64
parch            int64
fare           float64
embarked        object
class           object
who             object
adult_male        bool
deck            object
embark_town     object
alive           object
alone             bool
dtype: object

##### Utilizando o dicionário

In [6]:
import json

with open('../data/external/dict_titanic.json', 'r') as f:
    data_dict = json.load(f)

In [7]:
# Separando as variáveis quantitativas e qualitativas
variaveis_quantitativas = [var for var, meta in data_dict.items() if meta['tipo'] == 'quantitativa']
variaveis_qualitativas = [var for var, meta in data_dict.items() if meta['tipo'] == 'qualitativa']

print("Variáveis quantitativas:", variaveis_quantitativas)
print("Variáveis qualitativas:", variaveis_qualitativas)

Variáveis quantitativas: ['age', 'sibsp', 'parch', 'fare']
Variáveis qualitativas: ['survived', 'pclass', 'sex', 'embarked', 'class', 'who', 'adult_male', 'deck', 'embark_town', 'alive', 'alone']


#### Qtd de valores ausentes

In [8]:
valores_ausentes = df.isnull().sum()
valores_ausentes

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

##### Utilizando o dicionário

In [9]:
# Dicionário de dados com a quantidade de valores ausentes
for col in df.columns:
    descricao = data_dict.get(col, {}).get('descricao', 'Descrição não encontrada')
    tipo = data_dict.get(col, {}).get('tipo', 'Tipo não encontrado')
    subtipo = data_dict.get(col, {}).get('subtipo', 'Subtipo não encontrado')
    
    print(f"Variável: {col}")
    print(f"Descrição: {descricao}")
    print(f"Tipo: {tipo} ({subtipo})")
    print(f"Valores ausentes: {valores_ausentes[col]}\n")

Variável: survived
Descrição: Indica se o passageiro sobreviveu (1) ou não (0)
Tipo: qualitativa (nominal)
Valores ausentes: 0

Variável: pclass
Descrição: Classe do ticket do passageiro (1ª, 2ª, ou 3ª classe)
Tipo: qualitativa (ordinal)
Valores ausentes: 0

Variável: sex
Descrição: Sexo do passageiro (male ou female)
Tipo: qualitativa (nominal)
Valores ausentes: 0

Variável: age
Descrição: Idade do passageiro
Tipo: quantitativa (contínua)
Valores ausentes: 177

Variável: sibsp
Descrição: Número de irmãos/cônjuges a bordo
Tipo: quantitativa (discreta)
Valores ausentes: 0

Variável: parch
Descrição: Número de pais/filhos a bordo
Tipo: quantitativa (discreta)
Valores ausentes: 0

Variável: fare
Descrição: Valor da tarifa paga pelo passageiro
Tipo: quantitativa (contínua)
Valores ausentes: 0

Variável: embarked
Descrição: Porto de embarque (C = Cherbourg; Q = Queenstown; S = Southampton)
Tipo: qualitativa (nominal)
Valores ausentes: 2

Variável: class
Descrição: Classe socioeconômica do p

## 2. Perguntas de partida e hipóteses

#### Pergunta 1: Qual é a taxa de sobrevivência geral dos passageiros?

In [10]:
total_passageiros = df.shape[0]
sobreviventes = df['survived'].sum()
taxa_sobrevivencia = sobreviventes / total_passageiros * 100

print(f"Taxa de sobrevivência geral: {taxa_sobrevivencia:.2f}%")

Taxa de sobrevivência geral: 38.38%


#### Pergunta 2: O sexo dos passageiros teve impacto nas chances de sobrevivência?

In [11]:
sobrevivencia_sexo = df.groupby('sex')['survived'].mean() * 100

tx_sobrev_mulheres = sobrevivencia_sexo['female']
tx_sobrev_homens = sobrevivencia_sexo['male']

print("Taxa de sobrevivência por sexo:")
print(f"Mulheres: {tx_sobrev_mulheres:.2f}%")
print(f"Homens: {tx_sobrev_homens:.2f}%")

Taxa de sobrevivência por sexo:
Mulheres: 74.20%
Homens: 18.89%


#### Pergunta 3: A idade dos passageiros influenciou as chances de sobrevivência?

In [12]:
df['faixa_etaria'] = pd.cut(df['age'], bins=[0, 12, 18, 60, 80], labels=['Criança', 'Adolescente', 'Adulto', 'Idoso'])

sobrevivencia_idade = df.groupby('faixa_etaria', observed=False)['survived'].mean() * 100

tx_crianca = sobrevivencia_idade['Criança']
tx_adolescente = sobrevivencia_idade['Adolescente']
tx_adulto = sobrevivencia_idade['Adulto']
tx_idoso = sobrevivencia_idade['Idoso']

print("Taxa de sobrevivência por faixa etária:")
print(f"Criança: {tx_crianca:.2f}%")
print(f"Adolescente: {tx_adolescente:.2f}%")
print(f"Adulto: {tx_adulto:.2f}%")
print(f"Idoso: {tx_idoso:.2f}%")

Taxa de sobrevivência por faixa etária:
Criança: 57.97%
Adolescente: 42.86%
Adulto: 38.88%
Idoso: 22.73%


## 3. Insights

#### Taxa de sobrevivência geral

- Apenas uma minoria (38.38%) dos passageiros sobreviveu ao naufrágio, o que reflete a gravidade do desastre.
- Essa estatística básica serve como base para explorar outros fatores que podem ter influenciado as chances de sobrevivência.

#### Impacto do sexo da pessoa nas chances de sobrevivência

- Taxa de sobrevivência por sexo:
    - Mulheres: 74.20%
    - Homens: 18.89%
- Mulheres tiveram uma taxa de sobrevivência significativamente maior do que homens, o que pode refletir a política de "mulheres e crianças primeiro" adotada durante a evacuação.
- Esse insight reforça a ideia de que o sexo foi um fator determinante para as chances de sobrevivência no Titanic.

#### Influência da idade nas chances de sobrevivência

- Taxa de sobrevivência por faixa etária:
    - Criança: 57.97%
    - Adolescente: 42.86%
    - Adulto: 38.88%
    - Idoso: 22.73%
- A análise sugere que crianças tiveram uma taxa de sobrevivência mais alta, o que sugere que elas podem ter sido priorizadas durante o processo de evacuação.
- Em contrapartida, os idosos tiveram a menor taxa de sobrevivência, possivelmente devido à menor mobilidade e à dificuldade em acessar os botes salva-vidas, além de não terem recebido prioridade.